In [1]:
# Домашнее задание №9
# Исполнитель: Абдуллаев С. Ш.

# 1.На основе данных из таблицы track создайте столбчатую диаграмму, показывающую общую сумму минут (длительность треков) в разбивке по жанрам.
# 2.На основе данных из таблицы track создайте диаграмму рассеивания, показывающую соотношение минут и мегабайтов.
# 3.На основе данных из таблицы invoice создайте коробчатую диаграмму, показывающую распределение дохода (поле total) в разбивке по странам.
# 4.На основе данных из таблицы invoice создайте линейчатую диаграмму, показывающую изменение дохода по месяцам за 2025 год.
# 5.На основе данных из таблицы invoice, создайте круговую диаграмму, показывающую соотношение продаж в США и в остальном мире. Т.е. ваша круговая диаграмма должна состоять лишь из двух секций: США и остальной мир.

In [2]:
import psycopg2
import sqlalchemy as db
import numpy as np
import pandas as pd
import plotly.express as px
from sqlalchemy import text
from connector import connect_to

In [3]:
conn = connect_to('postgres')

In [4]:
# Задача 1: На основе данных из таблицы track создайте столбчатую диаграмму, показывающую общую сумму минут (длительность треков) в разбивке по жанрам.
query = """
    select 
     g."name"
     ,sum(t.milliseconds / 60000.) as duration_minutes 
    from 
     track t 
    join genre g 
    on t.genre_id = g.genre_id 
    group by
     g."name" ;;
"""

In [5]:

df = pd.read_sql(query, conn)
fig = px.bar(df, x='name', y='duration_minutes', title='Общая сумма минут по жанрам',
             labels={'name': 'Жанр', 'duration_minutes': 'Общая сумма минут'})
fig.show()

In [ ]:
#Задача 2: На основе данных из таблицы track создайте диаграмму рассеивания, показывающую соотношение минут и мегабайтов.
query = """
select 
   milliseconds / 60000. as duration_minutes
 , bytes / 1024 as size_mb
from
 track t
"""


In [21]:
df = pd.read_sql(query, conn)
fig = px.scatter(df, x='duration_minutes', y='size_mb', title='Соотношение минут и мегабайтов',
                 labels={'duration_minutes': 'Длительность (мин)', 'size_mb': 'Размер (МБ)'})
fig.show()

In [ ]:
# Задача 3: На основе данных из таблицы invoice создайте коробчатую диаграмму, показывающую распределение дохода (поле total) в разбивке по странам.

query = """

select 
 sum(total) as revenue, 
 billing_country as country
from
 invoice 
group by
 billing_country ;

"""

In [ ]:

df_invoice = pd.read_sql(query, conn)

fig3 = px.box(df_invoice, x='country', y='revenue', title='Распределение дохода по странам',
              labels = {'country' : 'Страна', 'revenue' : 'Выручка'})
fig3.show()

In [ ]:
#Задача 4: На основе данных из таблицы invoice создайте линейчатую диаграмму, показывающую изменение дохода по месяцам за 2025 год.

query = """
    select 
     date_trunc('month', invoice_date) as month
     ,total as revenue  
    from
      invoice i 
    where
      i.invoice_date >= '01.01.2025' 
      and i.invoice_date < '01.01.2026';
"""

In [39]:
df_invoice = pd.read_sql(query, conn)

monthly_revenue = df_invoice.groupby('month')['revenue'].sum().reset_index()
fig4 = px.line(monthly_revenue, x='month', y='revenue', title='Изменение дохода по месяцам за 2025 год')
fig4.show()

In [46]:
'''
 Задача 5: На основе данных из таблицы invoice, создайте круговую диаграмму, показывающую соотношение продаж в США и в остальном мире. 
           Т.е. ваша круговая диаграмма должна состоять лишь из двух секций: США и остальной мир.
'''
query = """
    select 
     billing_country as country,
     total as revenue
    from 
     invoice
"""


In [47]:
df_invoice = pd.read_sql(query, conn)
df_invoice['region'] = df_invoice['country'].apply(lambda x: 'USA' if x == 'USA' else 'Rest of the World')
sales_share = df_invoice.groupby('region')['revenue'].sum().reset_index()
fig5 = px.pie(sales_share, values='revenue', names='region', title='Соотношение продаж в США и в остальном мире')
fig5.show()